In [32]:
# 문서 로드
with open("9f9e37a6-55f4-43c4-8285-b4b976f5dd4b.md", "r", encoding="utf-8") as file:
    content = file.read()

print("문서 로드 완료")
print(f"문서 길이: {len(content)} 문자")
print(f"첫 100자: {content[:100]}")


문서 로드 완료
문서 길이: 15431 문자
첫 100자: # 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
| --- | --- |
| 든든급식 개편사항 안내(10


In [33]:
# 필요한 라이브러리 설치 및 로드
from langchain_text_splitters import RecursiveCharacterTextSplitter
from elasticsearch import Elasticsearch
import json

print("라이브러리 로드 완료")


라이브러리 로드 완료


In [34]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10,
    separators=["\n\n", "\n", " ", ""]
)

docs = text_splitter.create_documents([content])
print(f"문서가 {len(docs)}개의 chunk로 분할되었습니다.")


문서가 23개의 chunk로 분할되었습니다.


In [35]:
# Elasticsearch 연결 (Nori 플러그인 설치됨)
es = Elasticsearch(
    hosts=["http://localhost:9200"],
    verify_certs=False,
    ssl_show_warn=False
)

# 연결 및 플러그인 확인
if es.ping():
    print("✅ Elasticsearch 연결 성공!")
    
    # 플러그인 확인
    plugins = es.cat.plugins(format="json")
    nori_installed = any(plugin.get('component') == 'analysis-nori' for plugin in plugins)
    
    if nori_installed:
        print("✅ Nori 플러그인 설치 확인됨!")
    else:
        print("❌ Nori 플러그인이 설치되지 않았습니다.")
else:
    print("❌ Elasticsearch 연결 실패")


✅ Elasticsearch 연결 성공!
✅ Nori 플러그인 설치 확인됨!


In [36]:
# 먼저 간단한 Nori 테스트부터 시작
print("🧪 Nori 분석기 테스트 시작...")

# 가장 기본적인 Nori 설정으로 테스트
simple_nori_mapping = {
    "settings": {
        "analysis": {
            "analyzer": {
                "simple_korean": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "analyzer": "simple_korean"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 테스트 인덱스 생성
test_index = "nori_test"

if es.indices.exists(index=test_index):
    es.indices.delete(index=test_index)
    print("기존 테스트 인덱스 삭제")

try:
    es.indices.create(index=test_index, body=simple_nori_mapping)
    print("✅ 기본 Nori 인덱스 생성 성공!")
    
    # 간단한 텍스트 분석 테스트
    test_text = "장점"
    response = es.indices.analyze(
        index=test_index,
        body={"analyzer": "simple_korean", "text": test_text}
    )
    tokens = [token['token'] for token in response['tokens']]
    print(f"✅ 분석 테스트 성공: '{test_text}' → {tokens}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("기본 Standard 분석기로 대체합니다.")


🧪 Nori 분석기 테스트 시작...
기존 테스트 인덱스 삭제
✅ 기본 Nori 인덱스 생성 성공!
✅ 분석 테스트 성공: '장점' → ['장점']


In [37]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [38]:
# 모든 인덱스에 문서 인덱싱
indices_list = [standard_index, nori_index, ngram_index]

for index_name in indices_list:
    print(f"\n📁 '{index_name}' 인덱스에 문서 인덱싱 중...")
    
    for i, doc in enumerate(docs):
        document = {
            "content": doc.page_content,
            "chunk_id": i
        }
        es.index(index=index_name, id=i, body=document)
    
    es.indices.refresh(index=index_name)
    print(f"✅ {len(docs)}개 chunk 인덱싱 완료")

print("\n🎉 모든 인덱스 준비 완료!")



📁 'bm25_standard' 인덱스에 문서 인덱싱 중...
✅ 23개 chunk 인덱싱 완료

📁 'bm25_nori' 인덱스에 문서 인덱싱 중...
✅ 23개 chunk 인덱싱 완료

📁 'bm25_ngram' 인덱스에 문서 인덱싱 중...
✅ 23개 chunk 인덱싱 완료

🎉 모든 인덱스 준비 완료!


In [39]:
# 검색 함수 정의
def search_with_analyzer(query, index_name, top_k=3):
    """지정된 인덱스에서 BM25 검색 수행"""
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id"]
    }
    
    response = es.search(index=index_name, body=search_body)
    results = []
    
    for hit in response['hits']['hits']:
        results.append({
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content']
        })
    
    return results

print("검색 함수 정의 완료")


검색 함수 정의 완료


In [40]:
# 🏆 3가지 분석기 성능 비교 테스트
print("🏆 한글 BM25 성능 비교: Standard vs Nori vs N-gram")
print("=" * 80)

test_queries = [
    "서울든든급식",
    "식재료 주문", 
    "어린이집",
    "급식",
    "든든",
    "회원가입",
    "장점점"
]

for query in test_queries:
    print(f"\n🔍 검색어: '{query}'")
    print("=" * 60)
    
    # 1. Standard 분석기
    print("1️⃣ Standard 분석기:")
    standard_results = search_with_analyzer(query, standard_index, top_k=2)
    if standard_results:
        for i, result in enumerate(standard_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:70]}...")
    else:
        print("   검색 결과 없음")
    
    # 2. Nori 분석기 (한글 형태소 분석)
    print("\n2️⃣ Nori 분석기 (한글 형태소 분석):")
    nori_results = search_with_analyzer(query, nori_index, top_k=2)
    if nori_results:
        for i, result in enumerate(nori_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:70]}...")
    else:
        print("   검색 결과 없음")
    
    # 3. N-gram 분석기
    print("\n3️⃣ N-gram 분석기 (2-3글자 단위):")
    ngram_results = search_with_analyzer(query, ngram_index, top_k=2)
    if ngram_results:
        for i, result in enumerate(ngram_results, 1):
            print(f"   [{i}] Chunk {result['chunk_id']} - 점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:70]}...")
    else:
        print("   검색 결과 없음")
    
    print("\n" + "=" * 60)


🏆 한글 BM25 성능 비교: Standard vs Nori vs N-gram

🔍 검색어: '서울든든급식'
1️⃣ Standard 분석기:
   [1] Chunk 7 - 점수: 2.8021
       내용: 11월 20일(수)부터 '24년 12월분 식재료 주문이 가능합니다.

성동구 어린이집 대상 신규 회원가입 안내 >

회원가입 ...
   [2] Chunk 0 - 점수: 2.6964
       내용: # 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
|...

2️⃣ Nori 분석기 (한글 형태소 분석):
   [1] Chunk 0 - 점수: 5.0503
       내용: # 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
|...
   [2] Chunk 7 - 점수: 4.7894
       내용: 11월 20일(수)부터 '24년 12월분 식재료 주문이 가능합니다.

성동구 어린이집 대상 신규 회원가입 안내 >

회원가입 ...

3️⃣ N-gram 분석기 (2-3글자 단위):
   [1] Chunk 0 - 점수: 0.9590
       내용: # 든든히 먹고
튼튼히 크는
서울든든급식# SE♡UL M! SOUL# 서울특별시# < 진행 순서 >| 내 용 | 진 행 |
|...
   [2] Chunk 7 - 점수: 0.9590
       내용: 11월 20일(수)부터 '24년 12월분 식재료 주문이 가능합니다.

성동구 어린이집 대상 신규 회원가입 안내 >

회원가입 ...


🔍 검색어: '식재료 주문'
1️⃣ Standard 분석기:
   [1] Chunk 8 - 점수: 2.7022
       내용: 5

해당 식재료를 담을 수 있는
날짜 표출

* 오늘(주문일) 기준

# 7일 뒤부터 주문 가능ex) 오늘이 수요일이라면,
...
   [2] Chunk 7 - 점수: 2.2662
       내용: 

In [41]:
# 토크나이저 분석 결과 비교
print("\n🔍 토크나이저 분석 결과 비교")
print("=" * 60)

def analyze_text(text, analyzer_name, index_name):
    """텍스트 분석 결과 반환"""
    try:
        response = es.indices.analyze(
            index=index_name,
            body={"analyzer": analyzer_name, "text": text}
        )
        return [token['token'] for token in response['tokens']]
    except Exception as e:
        print(f"분석 실패: {e}")
        return []

sample_texts = [
    "서울든든급식",
    "식재료 주문 방법",
    "어린이집 대상 신규 회원가입"
]

for text in sample_texts:
    print(f"\n📝 분석 텍스트: '{text}'")
    print("-" * 40)
    
    # Standard 분석
    standard_tokens = analyze_text(text, "standard", standard_index)
    print(f"🔹 Standard: {standard_tokens}")
    
    # Nori 분석
    nori_tokens = analyze_text(text, "korean_analyzer", nori_index)
    print(f"🔹 Nori:     {nori_tokens}")
    
    # N-gram 분석 (일부만 표시)
    ngram_tokens = analyze_text(text, "ngram_analyzer", ngram_index)
    print(f"🔹 N-gram:   {ngram_tokens[:8]}{'...' if len(ngram_tokens) > 8 else ''}")
    
    print(f"💡 토큰 개수 - Standard: {len(standard_tokens)}, Nori: {len(nori_tokens)}, N-gram: {len(ngram_tokens)}")
    print("-" * 40)



🔍 토크나이저 분석 결과 비교

📝 분석 텍스트: '서울든든급식'
----------------------------------------
🔹 Standard: ['서울든든급식']
🔹 Nori:     ['서울', '든든', '급식']
🔹 N-gram:   ['서울', '서울든', '울든', '울든든', '든든', '든든급', '든급', '든급식']...
💡 토큰 개수 - Standard: 1, Nori: 3, N-gram: 9
----------------------------------------

📝 분석 텍스트: '식재료 주문 방법'
----------------------------------------
🔹 Standard: ['식재료', '주문', '방법']
🔹 Nori:     ['식', '재료', '주문', '방법']
🔹 N-gram:   ['식재', '식재료', '재료', '재료 ', '료 ', '료 주', ' 주', ' 주문']...
💡 토큰 개수 - Standard: 3, Nori: 4, N-gram: 15
----------------------------------------

📝 분석 텍스트: '어린이집 대상 신규 회원가입'
----------------------------------------
🔹 Standard: ['어린이집', '대상', '신규', '회원가입']
🔹 Nori:     ['어린이', '집', '대상', '신규', '회원', '가입']
🔹 N-gram:   ['어린', '어린이', '린이', '린이집', '이집', '이집 ', '집 ', '집 대']...
💡 토큰 개수 - Standard: 4, Nori: 6, N-gram: 27
----------------------------------------


In [42]:
# 📄 새로운 문서 추가: AX브릿지위원회 소개자료
print("📄 새로운 문서 로드 중...")

# 새로운 문서 로드
with open("ceca9bf3-6183-40bc-ad64-52911b2015b3.md", "r", encoding="utf-8") as file:
    new_content = file.read()

print(f"✅ 새 문서 로드 완료: {len(new_content)} 문자")
print(f"문서 제목: {new_content.split('\\n')[0][:50]}...")

# 새 문서도 동일한 방식으로 분할
new_docs = text_splitter.create_documents([new_content])
print(f"✅ 새 문서가 {len(new_docs)}개의 chunk로 분할되었습니다.")

# 기존 chunk 개수 확인
print(f"기존 문서 chunk 개수: {len(docs)}")
print(f"새 문서 chunk 개수: {len(new_docs)}")
print(f"총 chunk 개수: {len(docs) + len(new_docs)}")

# 전체 문서 리스트 결합
all_docs = docs + new_docs
print(f"\\n🎯 총 {len(all_docs)}개의 chunk로 확장되었습니다!")


SyntaxError: f-string expression part cannot include a backslash (983037729.py, line 9)

In [ ]:
# 🔄 기존 인덱스에 새로운 문서 추가
print("🔄 기존 인덱스에 새로운 문서 추가 중...")

# 기존 chunk 개수를 시작 ID로 사용
start_id = len(docs)

# 모든 인덱스에 새로운 문서 추가
for index_name in indices_list:
    print(f"\\n📁 '{index_name}' 인덱스에 새 문서 추가 중...")
    
    # 새로운 chunk들을 기존 인덱스에 추가
    for i, doc in enumerate(new_docs):
        document = {
            "content": doc.page_content,
            "chunk_id": start_id + i,  # 기존 ID에 이어서 번호 부여
            "source": "AX브릿지위원회"  # 출처 구분을 위한 필드 추가
        }
        es.index(index=index_name, id=start_id + i, body=document)
    
    # 인덱스 새로고침
    es.indices.refresh(index=index_name)
    print(f"✅ {len(new_docs)}개 새 chunk 추가 완료")

print(f"\\n🎉 모든 인덱스 업데이트 완료!")
print(f"📊 총 문서 수: {len(all_docs)}개 chunk")
print(f"   - 서울든든급식: {len(docs)}개")
print(f"   - AX브릿지위원회: {len(new_docs)}개")


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 3가지 인덱스 생성: Standard, Nori, N-gram

# 1. Standard 분석기 인덱스
standard_index = "bm25_standard"
standard_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "bm25_similarity": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "bm25_similarity", "analyzer": "standard"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 2. Nori 분석기 인덱스 (간단하고 안전한 설정)
nori_index = "bm25_nori"
nori_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "korean_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "korean_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "korean_bm25", "analyzer": "korean_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 3. N-gram 분석기 인덱스
ngram_index = "bm25_ngram"
ngram_mapping = {
    "settings": {
        "index": {
            "similarity": {
                "ngram_bm25": {"type": "BM25", "k1": 1.2, "b": 0.75}
            }
        },
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "keyword",
                    "filter": ["lowercase", "ngram_filter"]
                }
            },
            "filter": {
                "ngram_filter": {"type": "ngram", "min_gram": 2, "max_gram": 3}
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {"type": "text", "similarity": "ngram_bm25", "analyzer": "ngram_analyzer"},
            "chunk_id": {"type": "integer"}
        }
    }
}

# 인덱스 생성
indices = [(standard_index, standard_mapping), (nori_index, nori_mapping), (ngram_index, ngram_mapping)]

for index_name, mapping in indices:
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"기존 인덱스 '{index_name}' 삭제")
    
    es.indices.create(index=index_name, body=mapping)
    print(f"✅ 인덱스 '{index_name}' 생성 완료")


기존 인덱스 'bm25_standard' 삭제
✅ 인덱스 'bm25_standard' 생성 완료
기존 인덱스 'bm25_nori' 삭제
✅ 인덱스 'bm25_nori' 생성 완료
기존 인덱스 'bm25_ngram' 삭제
✅ 인덱스 'bm25_ngram' 생성 완료


In [ ]:
# 🔍 통합 검색 테스트 - 두 문서에서 동시 검색
print("🔍 통합 검색 테스트 시작!")
print("=" * 80)

# 확장된 검색 함수 - 출처 정보 포함
def search_with_source(query, index_name, top_k=5):
    """출처 정보를 포함한 검색 함수"""
    search_body = {
        "size": top_k,
        "query": {
            "match": {
                "content": {
                    "query": query,
                    "operator": "or"
                }
            }
        },
        "_source": ["content", "chunk_id", "source"]
    }
    
    response = es.search(index=index_name, body=search_body)
    results = []
    
    for hit in response['hits']['hits']:
        source_info = hit['_source'].get('source', '서울든든급식')  # 기본값은 서울든든급식
        results.append({
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content'],
            'source': source_info
        })
    
    return results

# 다양한 검색어로 통합 검색 테스트
mixed_queries = [
    "서울든든급식",      # 첫 번째 문서 관련
    "AX브릿지위원회",    # 두 번째 문서 관련
    "AI",               # 두 번째 문서 관련
    "벤처기업",         # 두 번째 문서 관련
    "메가존클라우드",   # 두 번째 문서 관련
    "급식",             # 첫 번째 문서 관련
    "위원회",           # 두 번째 문서 관련
    "대표이사"          # 두 번째 문서 관련
]

print("🏆 통합 검색 결과 (Nori 분석기 사용)")
print("=" * 80)

for query in mixed_queries:
    print(f"\\n🔍 검색어: '{query}'")
    print("=" * 60)
    
    # Nori 분석기로 검색
    results = search_with_source(query, nori_index, top_k=3)
    
    if results:
        for i, result in enumerate(results, 1):
            source_emoji = "🍽️" if result['source'] == "서울든든급식" else "🚀"
            print(f"   [{i}] {source_emoji} {result['source']} - Chunk {result['chunk_id']}")
            print(f"       점수: {result['score']:.4f}")
            print(f"       내용: {result['content'][:80]}...")
            print()
    else:
        print("   검색 결과 없음")
    
    print("-" * 60)


In [ ]:
# 📊 문서별 검색 통계 및 분석
print("\\n📊 문서별 검색 통계 분석")
print("=" * 60)

def analyze_search_distribution(query, index_name, top_k=10):
    """검색 결과의 문서별 분포 분석"""
    results = search_with_source(query, index_name, top_k=top_k)
    
    # 문서별 개수 계산
    source_count = {}
    for result in results:
        source = result['source']
        if source not in source_count:
            source_count[source] = 0
        source_count[source] += 1
    
    return results, source_count

# 주요 검색어들에 대한 분포 분석
analysis_queries = ["AI", "급식", "위원회", "대표", "서울"]

for query in analysis_queries:
    print(f"\\n🔍 '{query}' 검색 분포:")
    results, distribution = analyze_search_distribution(query, nori_index, top_k=10)
    
    total_results = len(results)
    if total_results > 0:
        print(f"   총 {total_results}개 결과:")
        for source, count in distribution.items():
            percentage = (count / total_results) * 100
            emoji = "🍽️" if source == "서울든든급식" else "🚀"
            print(f"   {emoji} {source}: {count}개 ({percentage:.1f}%)")
        
        # 상위 3개 결과 표시
        print(f"\\n   📋 상위 3개 결과:")
        for i, result in enumerate(results[:3], 1):
            source_emoji = "🍽️" if result['source'] == "서울든든급식" else "🚀"
            print(f"   [{i}] {source_emoji} 점수: {result['score']:.3f} - {result['content'][:50]}...")
    else:
        print("   검색 결과 없음")
    
    print("-" * 50)


In [ ]:
# 🎯 특정 문서에서만 검색하는 필터 기능
print("\n🎯 문서별 필터 검색 기능")
print("=" * 60)

def search_by_source(query, index_name, source_filter=None, top_k=5):
    """특정 출처 문서에서만 검색하는 함수"""
    if source_filter:
        # 특정 출처로 필터링 - source 필드가 keyword 타입이 아니므로 match 사용
        search_body = {
            "size": top_k,
            "query": {
                "bool": {
                    "must": [
                        {"match": {"content": {"query": query, "operator": "or"}}},
                        {"match": {"source": source_filter}}
                    ]
                }
            },
            "_source": ["content", "chunk_id", "source"]
        }
    else:
        # 전체 검색
        search_body = {
            "size": top_k,
            "query": {
                "match": {
                    "content": {
                        "query": query,
                        "operator": "or"
                    }
                }
            },
            "_source": ["content", "chunk_id", "source"]
        }
    
    response = es.search(index=index_name, body=search_body)
    results = []
    
    for hit in response['hits']['hits']:
        source_info = hit['_source'].get('source', '서울든든급식')
        results.append({
            'chunk_id': hit['_source']['chunk_id'],
            'score': hit['_score'],
            'content': hit['_source']['content'],
            'source': source_info
        })
    
    return results

# 필터 검색 테스트
test_query = "대표"

print(f"🔍 검색어: '{test_query}'\n")

# 1. 전체 검색
print("1️⃣ 전체 문서에서 검색:")
all_results = search_by_source(test_query, nori_index, source_filter=None, top_k=5)
for i, result in enumerate(all_results, 1):
    source_emoji = "🍽️" if result['source'] == "서울든든급식" else "🚀"
    print(f"   [{i}] {source_emoji} {result['source']} - 점수: {result['score']:.3f}")
    print(f"       {result['content'][:70]}...")

# 2. 서울든든급식에서만 검색
print("\n2️⃣ 서울든든급식 문서에서만 검색:")
seoul_results = search_by_source(test_query, nori_index, source_filter="서울든든급식", top_k=3)
if seoul_results:
    for i, result in enumerate(seoul_results, 1):
        print(f"   [{i}] 🍽️ 점수: {result['score']:.3f}")
        print(f"       {result['content'][:70]}...")
else:
    print("   검색 결과 없음")

# 3. AX브릿지위원회에서만 검색  
print("\n3️⃣ AX브릿지위원회 문서에서만 검색:")
ax_results = search_by_source(test_query, nori_index, source_filter="AX브릿지위원회", top_k=3)
if ax_results:
    for i, result in enumerate(ax_results, 1):
        print(f"   [{i}] 🚀 점수: {result['score']:.3f}")
        print(f"       {result['content'][:70]}...")
else:
    print("   검색 결과 없음")
